# I) tensors Basics

In [7]:
import tensorflow as tf
import numpy as np

#### Difference between numpy ndarray and tensors

In [28]:
x1 = np.arange(0,25)
print(x1)
x2 = tf.constant(x, dtype=tf.int32)
print(x2)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24]
tf.Tensor(
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24], shape=(25,), dtype=int32)


#### 1. There are two way to declare tensors
- tf.Variable(): mutable/changeable.
- tf.constant():  imutable/un-changeable.

In [19]:
tensor1 = tf.Variable(x1)
tensor2 = tf.constant(x2)
print(tensor1)
print(tensor2)

<tf.Variable 'Variable:0' shape=(25,) dtype=int32, numpy=
array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24])>
tf.Tensor(
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24], shape=(25,), dtype=int32)


#### 2. One feature of tensors is that they can be reshaped. When reshpaing, make sure you consider dimensions that will include all of the values of the tensor.

In [20]:
tensor1 = tf.reshape(tensor1, shape=(5,5))
tensor2 = tf.reshape(tensor2, shape=(5,5))
print(tensor1)
print(tensor2)

tf.Tensor(
[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]], shape=(5, 5), dtype=int32)
tf.Tensor(
[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]], shape=(5, 5), dtype=int32)


#### 3. You can define the shape in `tf.constant()`, but cant do that for `tf.Variable()`

In [29]:
try:
    # This will produce a ValueError
    tf.Variable([1,2,3,4], shape=(2,2))
except ValueError as v:
    # See what the ValueError says
    print(v)

temp2 = tf.constant([1,2,3,4], shape=(2,2))
print(temp2)

The initial value's shape ((4,)) is not compatible with the explicitly supplied `shape` argument ((2, 2)).
tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)


#### 4. You can also change data type of the values within the tensor.

In [31]:
x1 = tf.cast(temp2, dtype=tf.float32)
x2 = tf.cast(temp2, dtype=tf.int32)
print(x1)
print(x2)

tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)


#### 5. Basic Operations

### a. using tf functions.
- You can  print out only the values from the tensors using `tensor.numpy()` syntax.

In [43]:
x1, x2 = np.arange(0,10), np.arange(10, 20)
summ = tf.add(x1, x2)
sub = tf.subtract(x2, x1)
mul = tf.multiply(x1, x2)
div = tf.divide(x2, x1)
x1_sqr = tf.square(x1)
print(summ.numpy())
print(sub.numpy())
print(mul.numpy())
print(div.numpy())
print(x1_sqr.numpy())


[10 12 14 16 18 20 22 24 26 28]
[10 10 10 10 10 10 10 10 10 10]
[  0  11  24  39  56  75  96 119 144 171]
[        inf 11.          6.          4.33333333  3.5         3.
  2.66666667  2.42857143  2.25        2.11111111]
[ 0  1  4  9 16 25 36 49 64 81]


### b. using mathmetical operators

In [41]:
x1, x2 = tf.Variable(np.arange(0,10)), tf.Variable(np.arange(10, 20))

summ = x1 + x2
sub = x2 - x1
mul = x1 * x2
div = x1 / x2
print(summ.numpy())
print(sub.numpy())
print(mul.numpy())
print(div.numpy())

[10 12 14 16 18 20 22 24 26 28]
[10 10 10 10 10 10 10 10 10 10]
[  0  11  24  39  56  75  96 119 144 171]
[0.         0.09090909 0.16666667 0.23076923 0.28571429 0.33333333
 0.375      0.41176471 0.44444444 0.47368421]


# II) GradientTape() basics

### Basic Exercises

In [45]:
import tensorflow as tf
import numpy as np
w = tf.Variable([[2.0]])
with tf.GradientTape() as tape:
    loss = w**2

d_w = tape.gradient(loss, w)
print(d_w.numpy())

[[4.]]


In [46]:
x = tf.ones((2,2))
with tf.GradientTape() as tape:
    tape.watch(x)
    y = tf.reduce_sum(x)
    z = tf.square(y)
d_y= tape.gradient(z, x)
print(d_y.numpy())

[[8. 8.]
 [8. 8.]]


### Make the gradient tape persistent
To make sure that the gradient tape can be used multiple times, set `persistent=True` 

In [49]:
x = tf.Variable([1.0])
with tf.GradientTape() as tape:
    # Record the actions performed on tensor x with `watch`
    tape.watch(x)
    # Define y as the sum of the elements in x
    y = tf.reduce_sum(x)
    z = tf.square(y)
dz_dx = tape.gradient(z, x)
print(dz_dx.numpy())

[2.]


### Gradient tape expires after one use, by default

If you want to compute multiple gradients, note that by default, GradientTape is not persistent (`persistent=False`).  This means that the GradientTape will expire after you use it to calculate a gradient.

To see this, set up gradient tape as usual and calculate a gradient, so that the gradient tape will be 'expired'.

In [51]:
try:
    dy_dx = tape.gradient(y, x)
except RuntimeError as e:
    print(e)

A non-persistent GradientTape can only be used tocompute one set of gradients (or jacobians)


### Make the gradient tape persistent
To make sure that the gradient tape can be used multiple times, set `persistent=True` 

In [52]:
x = tf.Variable([1.0])
with tf.GradientTape(persistent=True) as tape:
    # Record the actions performed on tensor x with `watch`
    tape.watch(x)
    # Define y as the sum of the elements in x
    y = tf.reduce_sum(x)
    z = tf.square(y)
dz_dx = tape.gradient(z, x)
print(dz_dx.numpy())

[2.]


In [55]:
try:
    dy_dx = tape.gradient(y, x)
    print(dy_dx.numpy())
except RuntimeError as e:
    print(e)

[1.]


### Nested Gradient tapes
Now let's try computing a higher order derivative by nesting the `GradientTapes:`

#### Acceptable indentation of the first gradient calculation
Keep in mind that you'll want to make sure that the first gradient calculation of `dy_dx` should occur at least inside the outer `with` block.

### a. without the `persistent=True`, the tape will expire outside the scope, so it can't be used .

In [62]:
x = tf.Variable([3.0])

with tf.GradientTape() as tape2:
    with tf.GradientTape() as tape1:
        y = x**3
    dy_dx = tape1.gradient(y,x)
dy2_dx = tape2.gradient(dy_dx, x)

print(dy_dx.numpy())
print(dy2_dx.numpy())

[27.]
[18.]


### b.  by using, `persistent=True`.

In [64]:
x = tf.Variable([3.0])

with tf.GradientTape(persistent=True) as tape2:
    with tf.GradientTape(persistent=True) as tape1:
        y = x**3


dy_dx = tape1.gradient(y,x)
dy2_dx = tape2.gradient(dy_dx, x)

print(dy_dx)
print(dy2_dx)

tf.Tensor([27.], shape=(1,), dtype=float32)
None


#### Notice how the `d2y_dx2` calculation is now `None`.  The tape has expired.  Also note that this still won't work even if you set persistent=True for both gradient tapes.

### c. Proper indenting is needed.

In [68]:
x = tf.Variable([3.0])

with tf.GradientTape(persistent=True) as tape2:
    with tf.GradientTape(persistent=True) as tape1:
        y = x**3
        dy_dx = tape1.gradient(y,x)
    dy2_dx = tape2.gradient(dy_dx, x)

print(dy_dx.numpy())
print(dy2_dx.numpy())

[27.]
[18.]
